<a href="https://colab.research.google.com/github/MamadouBousso/Cours-Python/blob/main/TPContexte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gestionnaire de contexte

## Problémes

In [ ]:
#Si une exception survient durant l'ecriture rien ne garantit que le fichier sera  fermé
file = open("myfile.txt", "w")
file.write("Hello, World!")
file.close()

## Solutions avec try .... finally


In [ ]:
# Safely open the file
file = open("myfile.txt", "w")

try:
    file.write("Hello, World!")
finally:
    # Finally nous permet de nous assurer de la fermeture du fichier
    file.close()

In [ ]:
# Ou plus proprement ajouter un try except durant l'ecriture qui permet de capturer toutes les exceptions eventuelles
file = open("hello.txt", "w")

try:
    file.write("Hello, World!")
except Exception as e:
    print(f"An error occurred while writing to the file: {e}")
finally:
    # Finally nous permet de nous assurer de la fermeture du fichier
    file.close()

## Solutions avec with

In [ ]:
with expression as target_var:
    do_something(target_var)

NameError: ignored

In [ ]:
with open("myfile.txt", mode="w") as file:
    file.write("Hello, World!")

In [ ]:
with open("myfile.txt", mode="w") as file:
    file.write("Hello, World!")

file.write("Bonjour")

ValueError: ignored

In [ ]:
#Possibilité d'avoir plus contextes d'execution
with open("myfile.txt") as in_file, open("output.txt", "w") as out_file:
    # lire le contenu de input.txt
    contenu = in_file.read()
    # Transformer le contenu
    contenu = contenu + " ajout de texte"
    # Ecrire le contenu transforme dans output.txt
    out_file.write(contenu)
    #pass

In [ ]:
import os
# Parcourir les repertoires avec os.scandir qui retourne un iterateur sur objet os.DirEntry qui supporte le management context protocole
with os.scandir(".") as entries:
  
  for entry in entries:
    print(entry.name, "->", entry.stat().st_size, "bytes")

.config -> 4096 bytes
myfile.txt -> 13 bytes
output.txt -> 28 bytes
hello.txt -> 13 bytes
sample_data -> 4096 bytes


##  Problémes et exemples

In [ ]:
file = open("myfile.txt", mode="w")

with file:
  file.write("Bonjour, le monde!")


with file:
  file.write("Bienvenu au cours de Python!")

ValueError: ignored

Solution au probleme: utiliser [pathlib.Path.open](https://docs.python.org/3/library/pathlib.html#pathlib.Path.open)

In [ ]:
import pathlib

file_path = pathlib.Path("myfile.txt")

with file_path.open("w") as file:
  file.write("Bonjour, le monde!")

with file_path.open("w") as file:
  file.write("Bienvenu au cours de Python!")

Enfin, chaque fois que vous chargez un fichier externe, votre programme doit rechercher d'éventuels problèmes, tels qu'un fichier manquant, l'accès en écriture et en lecture, etc. Voici un modèle général que vous devriez envisager d'utiliser lorsque vous travaillez avec des fichiers:

In [ ]:
import pathlib
import logging

file_path = pathlib.Path("")

try:
    with file_path.open(mode="w") as file:
        file.write("Hello, World!")
except OSError as error:
    logging.error("Echec Ecriture dans le fichier %s due a: %s", file_path, error)

Exemple avec les decimaux
Il est possible d'augmenter la precision des décimaux dans un calcul et d'effectuer des calculs de haute precision avec [localcontext](https://docs.python.org/3/library/decimal.html#decimal.localcontext)

In [ ]:
from decimal import Decimal, localcontext

with localcontext() as ctx:
  ctx.prec = 52
  print(Decimal("5") / Decimal("42"))
  print(Decimal("83.3") - Decimal("100"))


Decimal("5") / Decimal("42")


In [ ]:
print(Decimal("83.3") - Decimal("100"))

## Classe personnalisée de gestion de contexte

In [ ]:
class GestionnaireContexte:
  def __enter__(self):
    print("J'entre dans le contexte")
    return self

  def __exit__(self,ctx_excpt,ctx_value,ctx_tb):
    print("Bye Bye je sors du contexte")
    print(ctx_excpt,ctx_value,ctx_tb)
    return ctx_value

  def afficher(self):
    print("....je travaille dans le contexte")
  


In [ ]:
with GestionnaireContexte() as gc:
  gc.afficher()

In [ ]:
with GestionnaireContexte() as gc:
  gc.afficher("Hello world")

**Ameliorons le code en gerant l'exception TypeError au cas où c'est la plus frequente**
Dans .__ exit __ (), vous vérifiez si ctx_value est une instance de 'TypeError'. Si tel est le cas, vous imprimez quelques messages informatifs et renvoyez finalement  True. Le renvoi d'une valeur de vérité permet d'avaler l'exception et de continuer l'exécution normale après le code du bloc with.

In [ ]:
class GestionnaireContexte2:
  def __enter__(self):
    print("J'entre dans le contexte")
    return self

  def __exit__(self,ctx_excpt,ctx_value,ctx_tb):
    print("Bye Bye je sors du contexte")
    if isinstance(ctx_value, TypeError):
      print(ctx_excpt,ctx_tb)
      return True
    return False


  def write(self):
    print("....je travaille dans le contexte")


In [ ]:
with GestionnaireContexte2() as gc:
  gc.write("test")

### Réutilisation

In [ ]:
ctx = GestionnaireContexte2()
with ctx:
  pass
 
with ctx:
  pass
  
with open("myfile.txt","w") as file:
  file.write("test")
with open("myfile.txt","w") as file:
  file.write("test")

### Réentrance


In [ ]:
with ctx:
  with ctx:
    with ctx:
      pass

In [ ]:
with GestionnaireContexte2():
  with GestionnaireContexte2():
    with GestionnaireContexte2():
      pass


**Probleme avec Lock de Threading qui est réutilisable mais pas réentrant**

In [ ]:
from threading import Lock
lock = Lock()
# Le bloc exterieur attend la fin du bloc interieur pour liberer la ressource
with lock:
  #le bloc intérieur demande l’accès à une ressource (lock) déjà occupée par le bloc extérieur
  with lock:
    pass
#Python est bloqué

In [ ]:
from threading import RLock
lock = RLock()

with lock:
  
  with lock:
    pass


**Correction exercice**

In [ ]:
class Indenter:
    def __init__(self):
        self.level = -1

    def __enter__(self):
        #print("J'entre dans le contexte")
        self.level += 1
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        #print("Bye Bye Je sors du contexte")
        self.level -= 1

    def print(self, text):
        print("***" * self.level + " "+text+" "+"***"* self.level)

In [ ]:
with Indenter() as indent:
  indent.print("Bonjour pas d'indentation")
  with indent as ind:
    ind.print(f"Bonjour indentation niveau {ind.level} avec des etoiles")
    with ind as i:
      i.print(f"Bonjour indentation niveau {i.level}")
  indent.print("Sortie")


In [ ]:
class OpenFile:
    def __init__(self, file_path,param):
        self.file_path = file_path
        self.param = param

    def __enter__(self):
        self.file_obj = open(self.file_path, mode= self.param)
        return self.file_obj

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file_obj:
            self.file_obj.close()

In [ ]:
with OpenFile("myfile.txt","w") as file:
  file.write("De nouveau present")

In [ ]:
with OpenFile("myfile.txt","r") as file:
  l = file.read()
  print(l)

## Utilisation de Contextlib

In [ ]:
# Creation d'une classe gestionnaire de contexte qui permet de mesurer les temps d'execution d'une suite d'instruction
from contextlib import ContextDecorator
import time

class SpentTime(ContextDecorator):
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, *_):
        print('Elapsed {:.3}s'.format(time.time() - self.start))

In [ ]:
from math import factorial
#On peut l'utiliser avec with
with SpentTime():
  factorial(9009)


In [ ]:
@SpentTime()
def factoriel(n):
  if n == 0:
    return 1
  else:
    return n*factoriel(n-1)

In [ ]:
factoriel(10)

In [ ]:
@SpentTime()
def factorielBis(n):
  P = 1
  for i in range(1,n+1):
    P = P*i
  return P

In [ ]:
factorielBis(10)

**Exercice sur la redirection**

In [ ]:
import sys
import contextlib as cl
class Redirection(cl.ContextDecorator):
    def __init__(self,fichier):
      self.fichier = fichier
      
    def __enter__(self):
        self.old_stdout = sys.stdout
        sys.stdout = self.fichier

    def __exit__(self, *_):
        sys.stdout = self.old_stdout
        

In [ ]:

output = open("myfile.txt","w")
with Redirection(output):
  print('ceci est écrit dans output et est verifiable')




In [ ]:
output = open("myfile.txt","w")
@Redirection(output)
def addition(a, b):
  print('result =', a + b)

In [ ]:
addition(2,3)

In [ ]:
#Réentrance
output = open("myfile.txt","w")
with Redirection(output):
  print('ceci est ecrit dans loutput1 ')
  
  
  with Redirection(output):
    print('ceci est ecrit dans loutput2 ')

print('ceci est ecrit dans quoi')
print('ceci est ecrit dans quoi 2')


In [ ]:
import sys
class RedirectionPile(cl.ContextDecorator):
    def __init__(self,fichier):
      self.fichier = fichier
      self.stack = []
    def __enter__(self):
        self.stack.append(sys.stdout)
        sys.stdout = self.fichier


    def __exit__(self, *_):
        sys.stdout = self.stack.pop()

In [ ]:
#Réentrance
output = open("myfile.txt","w")
with RedirectionPile(output):
  
  
  with RedirectionPile(output):
    print('ceci est ecrit dans la console ')
print('ceci est ecrit dans quoi')
print('ceci est ecrit dans quoi 2')


In [ ]:
def myxml(balise,contenu,** kwargs):
  attrs = ' '.join([' '+tup[0]+'='+f'"{tup[1]}"' for tup in list(kwargs.items())])
  return f"<{balise}{attrs}>{contenu}</{balise}>"

In [ ]:
myxml("balise","contenu",a = 1,b=2,c=3)

In [ ]:
def myxml2(balise,contenu,** kwargs):
  attrs = ' '.join([f' {cle} = "{valeur}"' for cle, valeur in kwargs.items()])
  return f"<{balise}{attrs}>{contenu}</{balise}>"

In [ ]:
myxml2("balise","contenu",a = 1,b=2,c=3)